# Aula 4 - Exercícios - Pandas, Fontes de dados e Agregação
### Consulta a internet (Google, Stackoverflow e etc) liberada!!

### 4.3 Escreva uma função executar uma query SQL de agregação num banco de dados SQLite que retorne a média dos valores de uma coluna VALORES agregados por uma coluna CATEGORIAS.
#### Dê o apelido Media para a média a métrica AVG da sua query. Ou seja: AVG(CATEGORIA) AS Media
#### Retorne a coluna CATEGORIA na tabela de resultados para identificar os valores. Ordene os resultados pelo valor da média em ordem decrescente.
#### Exemplo: Calcule média do IDHM (coluna VALORES) agregado por Ano da pesquisa (CATEGORIAS).

In [1]:
# Importe as bibliotecas pandas, numpy, glob e sqlalchemy
import pandas as pd
import numpy as np
import sqlalchemy as sqla

In [6]:
# Dica: A resposta deste exercício é bem semelhante ao código mostrado na aula, porém você
#       precisa parametrizar a string da query com os argumentos passados na função.
def soma_com_agregacao(val, cat, table, con_str):
    """retorne um dataframe contendo a soma dos valores em val dentro das categorias cat da tabela table
    """
    con = sqla.create_engine(con_str)
    return pd.read_sql_query(f"select {cat}, AVG({val}) as Media \
                            from {table} \
                            group by {cat} \
                            order by Media desc", con)

In [4]:
## Faça seus testes aqui
con_str='sqlite:///idhm2013.sqlite'

cat = table = 'IDHM'
val = 'ANO'
con = sqla.create_engine(con_str)
df = pd.read_sql_query(f"select {cat}, AVG({val}) as Media \
                            from {table} \
                            group by {cat} \
                            order by Media desc",
                       con
                      )
df.head()

,IDHM,Media
0,0.862,2010.0
1,0.854,2010.0
2,0.847,2010.0
3,0.845,2010.0
4,0.840,2010.0


In [7]:
# Validação para nota
con = sqla.create_engine(con_str)
path_idhm='https://github.com/alexlopespereira/curso_ciencia_dados2020/raw/master/data/originais/idhm/idhm2013.xlsx'
con_str='sqlite:///idhm2013.sqlite'
table_name='IDHM'
df_idhm = pd.read_excel(path_idhm, sheet_name="IDHM")
df_idhm.index.names = ['index']
df_idhm.to_sql(table_name, con, if_exists="replace")
test_data = {'ANO': {0: 2010, 1: 2000, 2: 1991}, 'Media': {0: 0.7, 1: 0.5, 2: 0.4}}
df_agg = soma_com_agregacao('IDHM', 'ANO', table_name, con_str)
df_agg['Media'] = df_agg['Media'].apply(lambda x: round(x, 1))
assert df_agg.to_dict() == test_data